In [1]:
# 3party
%config IPCompleter.greedy=True 
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import LeavePGroupsOut
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid
from sklearn import svm
from sklearn import preprocessing
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import numpy as np

# python
import os
from IPython.core.debugger import Tracer;

# classification packages-------------
from supervised_new import *
from prep_input_supervised import *
from plot_classification import *
from supervised_new import *
from parameter_search import *
from cross_validation import *
from save_for_class import *
# from plot_r_classification import *


In [2]:

path_to_save = '/Volumes/ASSD/pre_epi_seizures/h5_files/classification_datasets'
class_metadata = ['labels', 'group', 'sample_domain']

In [3]:
def get_data_struct(pt_list):
    # Select Patient to analyze
    # pt = 3
    global path_to_save
    path_to_save += '/patient_' + str(pt_list)
    
    
    data_struct = load_feature_groups_baseline_seizure_per_patient(
                                                    pt_list,
                                                    'hrv_time_features#'
                                                    )
    
    return data_struct

In [4]:
def make_dir(dir_):
    if not os.path.exists(dir_):
        print 'this '
        print 'MAKING'
        print dir_
        os.makedirs(dir_)
        print 'second this'


In [5]:
def general(file_to_save, down, up, data_struct, trial):
    
    data_struct = prep_input_supervised_baseline_seizure(data_struct, interval).reset_index()
    data_struct = data_struct.drop(['index'], axis=1)

    # explore_r(data_struct)

    # stop
    # print data_struct
    
    points_per_label = data_struct['labels'].reset_index().groupby(['labels']).count()

    points_per_label = list(dict(points_per_label)['index'])

    feature_names = [name
                     for name in data_struct.columns
                     if name not in class_metadata]

    # prepare data for classification - watch out for memory concerns
    X = data_struct.drop(class_metadata, axis=1)
    y = data_struct['labels']
    groups = data_struct['group']

    # Pipeline-------------------------------------------
    # choose normalization criterion
    scaler = preprocessing.StandardScaler()
    #X = scaler.transform(X) # apply
    #X = pd.DataFrame(X, columns=feature_names)

    # choose estimator
    model = 'SVC'
    estimator = svm.SVC() 
    
    # create pipeline object with all of the above steps
    pipe = Pipeline([('scaler', scaler), ('estimate', estimator)])
    #------------------------------------------------------
    
    # -->**insert here cross-validation strategy ( Testing nested LOGO )
    groups = data_struct['group']
    cv_out = LeavePGroupsOut(n_groups=1)
    cv_in = LeavePGroupsOut(n_groups=1)

    # choose param grid for exhaustive search
    # param_grid = {'C': (1, 10, 100, 1000), 'gamma': (0.001, 0.0001), 'kernel': ('rbf')}
    param_grid = [{'estimate__kernel': ['rbf'],
                   'estimate__gamma': [2**i for i in xrange(-15, -1)],
                   'estimate__C': [2**i for i in xrange(-5, 11)]},
                  ]
    
    # choose scoring
    scoring = {'AUC': 'roc_auc'}

    compute_all_new = False
    
    
    full_path = get_full_pipeline_name(path_to_save,
                                       file_to_save,
                                       pipe, 
                                       scoring,
                                       param_grid,
                                       feature_names,
                                       cv_out,
                                       cv_in,
                                       trial)

    # plot_scatter(path_to_save, data_struct, class_metadata)

    # plot_full(file_to_save, data_struct, class_metadata)
    print full_path
    make_dir(full_path)

    
    best_params = nested_cross_validation(full_path,
                                          X,y, groups,
                                          pipe,
                                          param_grid, scoring,
                                          feature_names, class_metadata,
                                          model,
                                          compute_all_new,
                                          cv_out,
                                          cv_in)
    
  

        
    path_ROC = full_path + '/ROC/'
    make_dir(path_ROC)
    #plot_hist(file_to_save, interval, points_per_label, data_struct, class_metadata)
    plot_roc_new(path_ROC, interval, points_per_label, best_params, trial)


In [6]:
data_struct = get_data_struct(pt_list = [3, 5])


mode = 'patient_semi_specific'

file_to_save = '/mode_' + str(mode) 

for step_min in xrange(5, 50, 5):

     file_to_save = file_to_save + '/step_' + str(step_min) + '/'

     for up, down in zip(xrange(step_min, 50, step_min), xrange(0, 50 - step_min, step_min)):
        interval = [(down*60, up*60)]

        file_to_save = file_to_save + 'parameter_optimization' + '_' + str(interval) + '/'
            
        trial_list = xrange(0, 1)
        
        for trial in trial_list:
            general(file_to_save, down, up, data_struct, trial)

[3, 5]


[2018-01-14 14:20:08.302566]:get_signal:20:Loading [signal: 3_FA7774DI_ECG-_2017-04-20 03:38:19.000000][group: /raw_$beginwin_samplerate:1000_win:0.001_init:0_finish:4200_endwin$__$beginparams_param:None_endparam$_/baseline_removal_$beginwin_win:0.001__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_filt:MedianFIR_endparam$_/rpeak_detection_$beginwin_win:rpeaks__samplerate:1000_endwin$__$beginparam_method:hamilton_endparam$_/hrv_computation_$beginwin_win:0.001__init:rpeaks[0]__finish:rpeaks[-1]__samplerate:1000_endwin$__$beginparam_method:hamilton__resampling:spline_endparam$_/hrv_time_features_$beginwin_win:120__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_method:hamilton__resampling:spline_endparam$_]
[2018-01-14 14:20:08.304710]:get_signal:20:Loading [signal: 5_FA7773TA_ECG-_2016-08-23 04:23:59.000000][group: /raw_$beginwin_samplerate:1000_win:0.001_init:0_finish:4200_endwin$__$beginparams_param:None_endparam$_/baseline_removal_$beginwin_win:0.001__init:0__fin

[('/raw_$beginwin_samplerate:1000_win:0.001_init:0_finish:4200_endwin$__$beginparams_param:None_endparam$_/baseline_removal_$beginwin_win:0.001__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_filt:MedianFIR_endparam$_/rpeak_detection_$beginwin_win:rpeaks__samplerate:1000_endwin$__$beginparam_method:hamilton_endparam$_/hrv_computation_$beginwin_win:0.001__init:rpeaks[0]__finish:rpeaks[-1]__samplerate:1000_endwin$__$beginparam_method:hamilton__resampling:spline_endparam$_/hrv_time_features_$beginwin_win:120__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_method:hamilton__resampling:spline_endparam$_', u'3_FA7774DI_ECG-_2017-04-20 03:38:19.000000'), ('/raw_$beginwin_samplerate:1000_win:0.001_init:0_finish:4200_endwin$__$beginparams_param:None_endparam$_/baseline_removal_$beginwin_win:0.001__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_filt:MedianFIR_endparam$_/rpeak_detection_$beginwin_win:rpeaks__samplerate:1000_endwin$__$beginparam_method:hamilton_endp

[2018-01-14 14:20:08.516213]:get_signal:20:Loading [signal: 3_FA7774DK_ECG-_2017-04-20 07:38:44.000000_0][group: /3000_1200/raw_$beginwin_samplerate:1000_win:0.001_init:0_finish:4200_endwin$__$beginparams_param:None_endparam$_/baseline_removal_$beginwin_win:0.001__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_filt:MedianFIR_endparam$_/rpeak_detection_$beginwin_win:rpeaks__samplerate:1000_endwin$__$beginparam_method:hamilton_endparam$_/hrv_computation_$beginwin_win:0.001__init:rpeaks[0]__finish:rpeaks[-1]__samplerate:1000_endwin$__$beginparam_method:hamilton__resampling:spline_endparam$_/hrv_time_features_$beginwin_win:120__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_method:hamilton__resampling:spline_endparam$_]
[2018-01-14 14:20:08.518452]:get_signal:20:Loading [signal: 3_FA7774DR_ECG-_2017-04-20 16:09:45.000000_1][group: /3000_1200/raw_$beginwin_samplerate:1000_win:0.001_init:0_finish:4200_endwin$__$beginparams_param:None_endparam$_/baseline_removal_$beginwi

[2018-01-14 14:20:08.677884]:get_signal:20:Loading [signal: 5_FA7773U1_ECG-_2016-08-25 01:44:21.000000_2][group: /3000_1200/raw_$beginwin_samplerate:1000_win:0.001_init:0_finish:4200_endwin$__$beginparams_param:None_endparam$_/baseline_removal_$beginwin_win:0.001__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_filt:MedianFIR_endparam$_/rpeak_detection_$beginwin_win:rpeaks__samplerate:1000_endwin$__$beginparam_method:hamilton_endparam$_/hrv_computation_$beginwin_win:0.001__init:rpeaks[0]__finish:rpeaks[-1]__samplerate:1000_endwin$__$beginparam_method:hamilton__resampling:spline_endparam$_/hrv_time_features_$beginwin_win:120__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_method:hamilton__resampling:spline_endparam$_]
[2018-01-14 14:20:08.825177]:get_signal:20:Loading [signal: window_3_FA7774DK_ECG-_2017-04-20 07:38:44.000000_0][group: /3000_1200/raw_$beginwin_samplerate:1000_win:0.001_init:0_finish:4200_endwin$__$beginparams_param:None_endparam$_/baseline_removal_$

...

[Parallel(n_jobs=1)]: Done 1120 out of 1120 | elapsed:   14.7s finished


Optimizing Hyper-parameters......
Fitting 5 folds for each of 224 candidates, totalling 1120 fits


[Parallel(n_jobs=1)]: Done 1120 out of 1120 | elapsed:   12.6s finished


Optimizing Hyper-parameters......
this 
MAKING
/Volumes/ASSD/pre_epi_seizures/h5_files/classification_datasets/patient_[3,5]/Model:('scaler',StandardScaler(copy=True,with_mean=True,with_std=True))/('estimate',SVC(C=1.0,cache_size=200,class_weight=None,coef0=0.0,decision_function_shape='ovr',degree=3,gamma='auto',kernel='rbf',max_iter=-1,probability=False,random_state=None,shrinking=True,tol=0.001,verbose=False))/Scoring:{'AUC':'roc_auc'}/Feature_Names:[u'mean_NN',u'SD_NN',u'p_NN50',u'var_NN',u'LF',u'HF',u'LF_HF']/CV_out:LeavePGroupsOut(n_groups=1)/CV_in:LeavePGroupsOut(n_groups=1)/Trial:0/mode_patient_semi_specific/step_5/parameter_optimization_[(0,300)]//ROC/
second this
{u'c': (0, 0.75, 0.75), u'b': (0, 0, 1), u'g': (0, 0.5, 0), u'k': (0, 0, 0), u'm': (0.75, 0, 0.75), u'r': (1, 0, 0), u'w': (1, 1, 1), u'y': (0.75, 0.75, 0)}
[({'estimate__gamma': 0.00390625, 'estimate__kernel': 'rbf', 'estimate__C': 0.03125},      mean_fit_time  mean_score_time  mean_test_AUC  mean_train_AUC  \
0     

NameError: global name 'stop' is not defined

In [ ]:
data_struct


In [ ]:
i = 'p  p  p'
i.replace(' ', '')

In [ ]:
clf